In [1]:
import sys
import random
sys.path.append('.')
import dataset

import numpy as np
from tqdm.auto import tqdm, trange

import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
import os
if not os.path.exists("../../data/data-train.csv.0"):
    dataset.create_n_splits("../../data/data-train.csv")

In [4]:
X_train, X_valid, X_test = dataset.load_datasets()

Caching dataset ../../data/data-train.csv.0 ...



Caching dataset ../../data/data-train.csv.1 ...



Caching dataset ../../data/data-train.csv.2 ...



Caching dataset ../../data/data-train.csv.3 ...



Caching dataset ../../data/sample-submission.csv ...



Caching dataset ../../data/data-train.csv.4 ...


In [5]:
# For final submission:
train_on_full_dataset = False
if train_on_full_dataset:
    X_train = X_train + X_valid

In [6]:
X_train.count_nonzero(), X_valid.count_nonzero(), X_test.count_nonzero()
valid_indices = list(set(zip(X_train.nonzero()[0], X_train.nonzero()[1])))

In [7]:
class Autoencoder(nn.Module):

    def __init__(self):
        super().__init__()
        self.user_dim = 10000
        self.item_dim = 1000
        
        self.layer_1 = nn.Linear(1000, 100)
        self.layer_1a = nn.Linear(100, 100)
        
        self.vae = nn.Linear(100, 100)
        
        self.layer_3 = nn.Linear(50, 100)
        self.cls_layer = nn.Linear(100, 1000)
    
    def forward(self, data):
        net_data = data

        # Encoder layers
        net_data = F.relu(self.layer_1(net_data))
        net_data = F.dropout(net_data, training=self.training)
        
        net_data = F.relu(self.layer_1a(net_data))
        net_data = F.dropout(net_data, training=self.training)
        
        # VAE bottleneck
        vae = self.vae(net_data)
        mus_q, log_sigmas_q = torch.split(vae, 50, dim=-1)
        stds_q = torch.exp(0.5 * log_sigmas_q)
        
        KL = 0.5 * (-log_sigmas_q + torch.exp(log_sigmas_q) + mus_q ** 2 - 1)
        KL = torch.sum(KL, dim=-1)
        KL = torch.mean(KL)
        
        # Sample random value if training,
        # use the mean during evaluation
        if not self.training:
            sampled_z = mus_q
        else:
            eps = torch.randn(stds_q.shape, dtype=torch.float, device=device)
            sampled_z = mus_q + eps * stds_q
        
        # Decoder layers
        net_data = F.relu(self.layer_3(sampled_z))
        net_data = F.dropout(net_data, training=self.training)
        
        # Classification layer
        y_score = self.cls_layer(net_data)
        return y_score, KL

In [8]:
# Model initialization
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
try:
    del model
except:
    pass
model = Autoencoder().to(device)
opt = torch.optim.Adam(model.parameters(), lr=3e-4)

In [9]:
print(X_train.count_nonzero(), X_valid.count_nonzero())

def get_predictions(A):
    """Predict the full matrix using the autoencoder."""
    model.eval()
    A_pred = np.zeros((10000, 1000))
    valid_indices = list(set(zip(A.nonzero()[0], A.nonzero()[1])))
    
    # Split dataset in batches of 64 rows ...
    for i in range(0, 10000, 64):
        X = X_train[i:i+64].todense()
        X_ = torch.tensor(X, dtype=torch.float, device=device)
        y_preds, KL = model(X_)
        
        for i_ in range(y_preds.shape[0]):
            x_i = y_preds[i_].detach().cpu()
            A_pred[i+i_] = x_i
    
    return A_pred, valid_indices

def compute_loss():
    """Compute the mean squared error between the ranking matrix and predictions."""
    A_pred, valid_indices = get_predictions(X_valid)
    losses = np.square(X_valid - A_pred)
    losses = [losses[i,j] for (i, j) in valid_indices]
    mean_loss = np.mean(losses)
    return mean_loss

compute_loss()

925567 251385


16.037534986267808

In [10]:
# Cache positions of nonempty indices for training

train_indices_perline = [[] for i_ in range(10000)]
for i, j in valid_indices:
    train_indices_perline[i].append((i, j))

for line in train_indices_perline:
    assert line
assert len(train_indices_perline) == 10000

In [11]:
ANNEAL_ALPHA = 0.0
NOISE_SCALE = 0.5
MAX_STEPS = 500000

loss_avg = []

for i in range(0, MAX_STEPS):
    # randomly sample 32 points from training matrix
    model.train()
    random_lines = random.sample(range(0, 10000), k=32)
    X = X_train[random_lines].todense()
    X = torch.tensor(X, dtype=torch.float, device=device)
    y_true = X
    
    opt.zero_grad()
    # Important: Add random noise to the input to prevent overfitting
    y_preds, KL = model(X + torch.randn(X.shape, device=device) * NOISE_SCALE)
    
    valid_mask = torch.zeros_like(y_preds, device=device)
    for i_ in range(X.shape[0]):
        i_real = random_lines[i_]
        indices = [j for _,j in train_indices_perline[i_real]]
        valid_mask[i_, indices] = 1.0

    loss = (y_preds - y_true) ** 2
    loss = loss[valid_mask != 0.]
    neg_ELBO = loss + ANNEAL_ALPHA * KL
    
    # update avg loss
    loss_avg.insert(0, loss.mean().item())
    loss_avg = loss_avg[:1000]
    
    if i % 1000 == 0:
        print("It %d Avg loss: %.4f" % (i, np.mean(loss_avg)))
    if i and i % 1000 == 0:
        mean_loss = compute_loss()
        print("Mean loss: %.4f" % mean_loss)
    
    neg_ELBO.mean().backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
    opt.step()


It 0 Avg loss: 16.2103
It 1000 Avg loss: 3.7257
Mean loss: 1.6864
It 2000 Avg loss: 1.5205
Mean loss: 1.1628
It 3000 Avg loss: 1.2870
Mean loss: 1.0507
It 4000 Avg loss: 1.2396
Mean loss: 1.0978
It 5000 Avg loss: 1.2203
Mean loss: 1.0435
It 6000 Avg loss: 1.2128
Mean loss: 1.0473
It 7000 Avg loss: 1.1953
Mean loss: 1.0666
It 8000 Avg loss: 1.1910
Mean loss: 1.0543
It 9000 Avg loss: 1.1856
Mean loss: 1.0421
It 10000 Avg loss: 1.1771
Mean loss: 1.0431
It 11000 Avg loss: 1.1758
Mean loss: 1.0432
It 12000 Avg loss: 1.1736
Mean loss: 1.0652
It 13000 Avg loss: 1.1674
Mean loss: 1.0448
It 14000 Avg loss: 1.1608
Mean loss: 1.0384
It 15000 Avg loss: 1.1661
Mean loss: 1.0405
It 16000 Avg loss: 1.1597
Mean loss: 1.0367
It 17000 Avg loss: 1.1577
Mean loss: 1.0477
It 18000 Avg loss: 1.1513
Mean loss: 1.0445
It 19000 Avg loss: 1.1498
Mean loss: 1.0560
It 20000 Avg loss: 1.1475
Mean loss: 1.0458
It 21000 Avg loss: 1.1433
Mean loss: 1.0398
It 22000 Avg loss: 1.1397
Mean loss: 1.0435
It 23000 Avg loss:

It 185000 Avg loss: 0.9867
Mean loss: 0.9855
It 186000 Avg loss: 0.9848
Mean loss: 0.9862
It 187000 Avg loss: 0.9874
Mean loss: 0.9870
It 188000 Avg loss: 0.9880
Mean loss: 0.9854
It 189000 Avg loss: 0.9861
Mean loss: 0.9858
It 190000 Avg loss: 0.9890
Mean loss: 0.9864
It 191000 Avg loss: 0.9837
Mean loss: 0.9872
It 192000 Avg loss: 0.9859
Mean loss: 0.9849
It 193000 Avg loss: 0.9848
Mean loss: 0.9851
It 194000 Avg loss: 0.9881
Mean loss: 0.9851
It 195000 Avg loss: 0.9808
Mean loss: 0.9859
It 196000 Avg loss: 0.9859
Mean loss: 0.9872
It 197000 Avg loss: 0.9834
Mean loss: 0.9847
It 198000 Avg loss: 0.9810
Mean loss: 0.9860
It 199000 Avg loss: 0.9854
Mean loss: 0.9857
It 200000 Avg loss: 0.9841
Mean loss: 0.9871
It 201000 Avg loss: 0.9853
Mean loss: 0.9862
It 202000 Avg loss: 0.9803
Mean loss: 0.9838
It 203000 Avg loss: 0.9846
Mean loss: 0.9846
It 204000 Avg loss: 0.9808
Mean loss: 0.9835
It 205000 Avg loss: 0.9852
Mean loss: 0.9828
It 206000 Avg loss: 0.9847
Mean loss: 0.9845
It 207000 

Mean loss: 0.9762
It 368000 Avg loss: 0.9594
Mean loss: 0.9771
It 369000 Avg loss: 0.9609
Mean loss: 0.9784
It 370000 Avg loss: 0.9557
Mean loss: 0.9771
It 371000 Avg loss: 0.9585
Mean loss: 0.9770
It 372000 Avg loss: 0.9605
Mean loss: 0.9766
It 373000 Avg loss: 0.9586
Mean loss: 0.9771
It 374000 Avg loss: 0.9587
Mean loss: 0.9776
It 375000 Avg loss: 0.9603
Mean loss: 0.9772
It 376000 Avg loss: 0.9579
Mean loss: 0.9781
It 377000 Avg loss: 0.9603
Mean loss: 0.9769
It 378000 Avg loss: 0.9599
Mean loss: 0.9759
It 379000 Avg loss: 0.9573
Mean loss: 0.9761
It 380000 Avg loss: 0.9558
Mean loss: 0.9758
It 381000 Avg loss: 0.9562
Mean loss: 0.9763
It 382000 Avg loss: 0.9568
Mean loss: 0.9762
It 383000 Avg loss: 0.9623
Mean loss: 0.9773
It 384000 Avg loss: 0.9592
Mean loss: 0.9761
It 385000 Avg loss: 0.9578
Mean loss: 0.9776
It 386000 Avg loss: 0.9557
Mean loss: 0.9764
It 387000 Avg loss: 0.9569
Mean loss: 0.9781
It 388000 Avg loss: 0.9588
Mean loss: 0.9763
It 389000 Avg loss: 0.9604
Mean loss:

In [14]:
mean_loss = compute_loss()
print(mean_loss)

0.9745764841373259


In [15]:
# The predicted matrix can also be used as initialization for other models ...
import pickle
with open("array-for-svd.pkl", "wb") as wp:
    pickle.dump(get_predictions(X_train)[0], wp)

In [ ]:
# Submission

In [16]:
import pandas as pd

def export_and_save(target, preds):
    target_rows, target_cols = target.nonzero()
    ids = [f"r{row+1}_c{col+1}" for row, col in zip(target_rows, target_cols)]
    scores = [preds[row, col] for row, col in zip(target_rows, target_cols)]
    # Clip scores out of valid range
    scores = [score if score <= 5.0 else 5.0 for score in scores]
    scores = [score if score >= 1.0 else 1.0 for score in scores]
    df = pd.DataFrame({"Id": ids, "Prediction": scores})
    print("---Please check---")
    print(df.head())
    df.to_csv("../../predictions/vae-1.csv", index=False)

export_and_save(X_test, get_predictions(X_test)[0])

---Please check---
        Id  Prediction
0    r1_c4    3.627194
1    r1_c8    3.384989
2   r1_c21    2.961046
3  r1_c102    3.931744
4  r1_c127    3.237733


In [17]:
# Check whether we exported the array correctly ...
import pickle
with open("array-for-svd.pkl", "rb") as fp:
    arr = pickle.load(fp)
    print(arr.shape, arr[0,7])
    del arr

(10000, 1000) 3.384988784790039
